# Network Visualizations: Dash

http://dash.plotly.com/interactive-graphing

https://dash.plotly.com/sharing-data-between-callbacks

https://medium.com/analytics-vidhya/interactive-visualization-with-plotly-and-dash-f3f840b786fa

https://dash-gallery.plotly.host/dash-cytoscape-lda/

(or without Plotly http://jonathansoma.com/lede/algorithms-2017/classes/networks/networkx-graphs-from-source-target-dataframe/)

In [1]:
import re, json, warnings
import pandas as pd
import numpy as np
import networkx as nx
from networkx.readwrite import json_graph
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import nxviz as nxv

# Import (Jupyter) Dash -- App Functionality
import dash
from dash.dependencies import Input, Output
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash

# Ignore simple warnings.
warnings.simplefilter('ignore', DeprecationWarning)

# Declare directory location to shorten filepaths later.
abs_dir = "/Users/quinn.wi/Documents/SemanticData/"

## App -- Dash + Plotly

In [ ]:
%%time

with open(abs_dir + "Output/Graphs/JQA_Network_mergedEntities-correlation/network.json",
          "r") as f:
    G = json.load(f)
    
G = json_graph.node_link_graph(G, directed = True)


# external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

# For vanilla dash, use 'dash.Dash(__name__)'
app = JupyterDash(__name__)#, external_stylesheets = external_stylesheets)


fig = go.Figure()

app.layout = html.Div([
    html.Div([
        html.I("Node Selection & Steps Away (press enter after changing name or steps)"),
        html.Br(),
        dcc.Input(id = 'node-select', type = 'search', value = 'adams george', debounce=True), # Autocomplete possible?
        dcc.Input(id = 'ego-steps', type = 'number', value = 1),
        dcc.RangeSlider(id='edge-range', min=0, max=1, step=0.01, value=[0.9, 1],
                        marks = {0: '0', 0.25:'0.25', 0.5: '0.5', 0.75:'0.75', 1:'1'}),
        html.Div(id="output-print"),
        html.Br(),
    ]),
    
    html.Div([
        dcc.Graph(id='graph-output', figure = fig)
    ])
    
])

# Print input values with updates.
@app.callback(
    Output("output-print", "children"),
    [Input("node-select", "value"), Input('ego-steps', 'value'), Input('edge-range', 'value')]
)
def update_output(nodeSelect, steps, edge_range):
        return f'Ego Node: {nodeSelect}, Steps Away from Ego: {steps}, Range of Edge Weight: {edge_range}'

    
# Use input values to update (subset) graph data.
@app.callback(
    Output('graph-output', 'figure'),
    [Input("node-select", "value"), Input('ego-steps', 'value'), Input('edge-range', 'value')]
)
def update_graph_data(nodeSelect, steps, edge_range):
    ego = nx.ego_graph(G, nodeSelect, radius = steps)
    
    lower_horizon = [(u, v, w) for (u, v, w) in ego.edges.data('weight') if w < edge_range[0]]
    upper_horizon = [(u, v, w) for (u, v, w) in ego.edges.data('weight') if w >= edge_range[1]]
    
    ego.remove_edges_from(lower_horizon + upper_horizon)
    
#     Create Network Graph with Plotly

        # Assign graph positions for each node.
    pos = nx.spring_layout(ego, k=0.5, iterations=50) # nx.layout.circular_layout

    for n, p in pos.items():
        ego.nodes[n]['pos'] = p

    # Create 'data' of edges for scatterplot.
    edge_x = []
    edge_y = []
    for edge in ego.edges():
        x0, y0 = ego.nodes[edge[0]]['pos']
        x1, y1 = ego.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    # Create 'trace' of scatter plot.   
    edge_trace = go.Scatter(
        x = edge_x, y = edge_y,
        line = dict(width = 0.5, color = '#888'),
        hoverinfo = 'none',
        mode = 'lines')

    # Create 'data' of nodes.
    node_x = []
    node_y = []
    node_degree = []
    node_modularity = []
    node_label = []
    for node in ego.nodes():
        x, y = ego.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)
        node_degree.append(ego.nodes[node]['degree'])
        node_modularity.append(ego.nodes[node]['modularity'])
        node_label.append(node)

    # Create 'trace' of nodes.
    node_trace = go.Scatter(
        x = node_x, y = node_y,
        mode = 'markers',
        hoverinfo = 'text',
        marker = dict(
            showscale = True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale = 'Rainbow',
            reversescale = False,
            color = node_modularity,
            size = 10,
            colorbar = dict(
                thickness = 15,
                title = 'Node Community',
                xanchor = 'left',
                titleside = 'right'
            ),
            line_width = 2))
    
    
#     Creage visualization fith Plotly.go
    fig = go.Figure(data = [edge_trace, node_trace],
                    layout = go.Layout(
                        title = f'{nodeSelect} Ego Network',
                        titlefont_size = 16,
                        showlegend = False,
                        hovermode = 'closest',
                        margin = dict(b = 20, l = 5, r = 5, t = 40),
                        annotations = [ dict(
                            text = "",
                            showarrow = True,
                            xref = "paper", yref = "paper",
                            x = 0.005, y = -0.002 ) ],
                        xaxis = dict(showgrid = False, zeroline = False, showticklabels = False),
                        yaxis = dict(showgrid = False, zeroline = False, showticklabels = False),
                        paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)')
                    )

    return fig

if __name__ == "__main__":
    app.run_server(mode = 'inline', debug = True) # mode = 'inline' for JupyterDash

## App -- Dash + Plotly + Pandas

#### Declare Functions

In [2]:
%%time


def create_nodes_edges_dfs(networkx_graph, ego_node, weight_min, weight_max):
    
    G = json_graph.node_link_graph(networkx_graph, directed = True)
    
    df = nx.to_pandas_edgelist(G)

    # Create first-degree edges & 'step' for node size.
    first_degree = df.query('(source == @ego_node) & (@weight_min < weight <= @weight_max)')
    first_degree.loc[:,'step'] = 20
    first_degree.loc[:,'color'] = '#ff4d4d'

    # Create second-degree edges.
    # Rule: a source in df that is a target in 'first_degree' = a second degree.
    # That is, it is the target of first_degree nodes.
    second_degree = df.loc[df['source'].isin(first_degree['target'].tolist())] \
        .query('(@weight_min <weight <= @weight_max)')
    second_degree['step'] = 10
    second_degree['color'] = '#ffe6e6'
    
    # Join first- & second-degree edges.
    edges = pd.concat([first_degree, second_degree])

    # Create nodes.
    nodes = set(edges['source'].tolist() + edges['target'].tolist() + [ego_node])
    nodes = pd.DataFrame(nodes, columns = ['target']) # 'target' because 'step' info about target, not source.


    # Merge node information with step.
    nodes = pd.merge(nodes,
                     edges[['target', 'step', 'color']].drop_duplicates(),
                     on = 'target', how = 'inner') \
        .rename(columns = {'target':'label'})

    # Create label + step for ego.
    nodes = nodes.append({'label':ego_node, 'step':40, 'color':'#800000'},
                         ignore_index = True)
    
    return nodes, edges

# https://blog.datasciencedojo.com/network-theory-game-of-thrones/
def make_graph(nodes_df, edges_df):
    g = nx.Graph()

    for i,row in nodes_df.iterrows():
        keys = row.index.tolist()
        values = row.values
        # The dict contains all attributes
        g.add_node(row['label'], **dict(zip(keys,values)))

    for i,row in edges_df.iterrows():
        keys = row.index.tolist()
        values = row.values
        g.add_edge(row['source'], row['target'], **dict(zip(keys,values)))
    
    return g

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs


#### Run App

In [3]:
%%time

with open(abs_dir + "Output/Graphs/JQA_Network_mergedEntities-correlation/network.json",
          "r") as f:
    G = json.load(f)


# external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

# For vanilla dash, use 'dash.Dash(__name__)'
app = JupyterDash(__name__)#, external_stylesheets = external_stylesheets)


fig = go.Figure()

app.layout = html.Div([
    html.Div([
        html.I("Node Selection (press enter after changing name)"),
        html.Br(),
        dcc.Input(id = 'node-select', type = 'search', value = 'adams george', debounce=True), # Autocomplete possible?
        dcc.RangeSlider(id='edge-range', min=0, max=1, step=0.01, value=[0.6, 1],
                        marks = {0: '0', 0.25:'0.25', 0.5: '0.5', 0.75:'0.75', 1:'1'}),
        html.Div(id="output-print"),
        html.Br(),
    ]),
    
    html.Div([
        dcc.Graph(id='graph-output', figure = fig)
    ])
    
])

# Print input values with updates.
@app.callback(
    Output("output-print", "children"),
    [Input("node-select", "value"), Input('edge-range', 'value')]
)
def update_output(nodeSelect, edge_range):
        return f'Ego Node: {nodeSelect}, Range of Edge Weight: {edge_range}'

    
# Use input values to update (subset) graph data.
@app.callback(
    Output('graph-output', 'figure'),
    [Input("node-select", "value"), Input('edge-range', 'value')]
)
def update_graph_data(nodeSelect, edge_range):
    nodes, edges = create_nodes_edges_dfs(G, nodeSelect, edge_range[0], edge_range[1])

    ego = make_graph(nodes, edges)

#     Assign graph positions for each node.
    pos = nx.spring_layout(ego, k=0.5, iterations=50) # nx.layout.circular_layout

    for n, p in pos.items():
        ego.nodes[n]['pos'] = p

        
    #     Create Network Graph with Plotly
        
    # Create 'data' of edges for scatterplot.
    edge_x = []
    edge_y = []
    edge_weight = []
    for edge in ego.edges():
        x0, y0 = ego.nodes[edge[0]]['pos']
        x1, y1 = ego.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
        edge_weight.append(ego.edges[edge]['weight'])

    # Create 'trace' of scatter plot.   
    edge_trace = go.Scatter(
        x = edge_x, y = edge_y,
        line = dict(width = 0.5,
                    color = '#888',
#                     width = edge_weight,
#                     color = node_color,
                    ),
        hoverinfo = 'none',
        mode = 'lines')
    
#     edge_trace['line'] = dict(width = edge_weight, color = '#888')

    # Create 'data' of nodes.
    node_x = []
    node_y = []
    node_size = []
    node_color = []
    node_label = []
    for node in ego.nodes():
        x, y = ego.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)
        node_size.append(ego.nodes[node]['step'])
        node_color.append(ego.nodes[node]['color'])
        node_label.append(node)

    # Create 'trace' of nodes.
    node_trace = go.Scatter(
        x = node_x, y = node_y,
        mode = 'markers',
        text = node_label, textposition = 'top center',
        hoverinfo = 'text',
        marker = dict(
            color = node_color,
            size = node_size,
            opacity = 1,
            line = dict(width = 1, color = '#330000'))
    )


    #     Creage visualization fith Plotly.go
    fig = go.Figure(data = [edge_trace, node_trace],
                    layout = go.Layout(
                        title = f'{nodeSelect} Ego Network',
                        titlefont_size = 16,
                        showlegend = False,
                        hovermode = 'closest',
                        margin = dict(b = 20, l = 5, r = 5, t = 40),
                        annotations = [ dict(
                            text = "",
                            showarrow = True,
                            xref = "paper", yref = "paper",
                            x = 0.005, y = -0.002 ) ],
                        xaxis = dict(showgrid = False, zeroline = False, showticklabels = False),
                        yaxis = dict(showgrid = False, zeroline = False, showticklabels = False),
                        paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)')
                    )

    return fig

if __name__ == "__main__":
    app.run_server(mode = 'inline', debug = True) # mode = 'inline' for JupyterDash

CPU times: user 115 ms, sys: 38.7 ms, total: 153 ms
Wall time: 300 ms


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## TO DO

1. Change Edge Weight (opacity/thickness) & Color